# **Análise de Sentimentos de _Reviews_ de produtos na plataforma Steam**

## **Importando bibliotecas**

In [1]:
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import numpy as np 
import re
import string
import unidecode

In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/juan-
[nltk_data]     burtet/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## **Carregando _dataset_** 

In [3]:
data = pd.read_csv('dataset/steam_reviews.tsv', sep='\t')[['review', 'voted_up']]

In [4]:
data.describe()

,review,voted_up
count,507051,507052
unique,506693,2
top,Err:507,True
freq,226,460579


In [5]:
data.dtypes

review      object
voted_up      bool
dtype: object

In [6]:
data.head(5)

,review,voted_up
0,"Corrupted é um jogo de RPG bem simplificado, o...",True
1,Meu PC tem uma placa de vídeo equivalente a GT...,False
2,Muito Bom! O meu diz que é recomendado specs:...,True
3,Funciona,True
4,"Muito bom o Steam VR, Very good.",True


### **Separando Exemplos**

#### Selecionando apenas Exemplos Positivos

In [7]:
positive = data[data['voted_up'] == True].sample(46000)
positive.head(5)

,review,voted_up
259074,"Sou um jogador muito ruim, demorei cerca de 10...",True
343190,"me senti mal jogando, faz tempo que um jogo nã...",True
291218,"Um clássico do jeitinho que eu esperava, uma n...",True
317220,Um bom jogo pra quem está querendo passar o te...,True
317647,UM OTIMO GAME MAS DIFICIL PRA PORRA,True


#### Selecionando apenas Exemplos Negativos

In [8]:
negative = data[data['voted_up'] == False].sample(46000)
negative.head(5)

,review,voted_up
274978,graphics look like sh[i][/i]it and everyones a...,False
482251,Antigamente o jogo era melhor...,False
113647,esperava mais de um jogo de 28 reais... mecani...,False
367157,"Viciante no começo, mas depois perde a graça.",False
187913,"Humble na moral, se for pra dar bosta de graça...",False


#### Concatenando os dados e misturando-os

In [9]:
frames = [positive, negative]
data = pd.concat(frames)
data = data.sample(frac=1).reset_index(drop=True)
data.head(10)

,review,voted_up
0,Apenas sensacional. Você no começo não dá nada...,True
1,"Muito bom, recomendo a todos os fãs q gostam d...",True
2,Um dos melhores RPG que jah joguei.. realmente...,True
3,Super recomendado. Jogo com uma jogabilidade ...,True
4,O jogo roda em slowmotion e mesmo procurando s...,False
5,"Eu recomendaria esse jogo,pois é um jogo criad...",True
6,"Gosto muito de action RPG`s, e me esforcei bas...",False
7,I'm a MLG PRO PLAYER QUICKSCOPER 720º NO SCOPE...,False
8,"Nada melhorou nos gráficos, o jogo apenas foi ...",False
9,Não joguem este game pois vai levar ban aleato...,False


## **Processando os Dados**

In [10]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('portuguese')).union(stopwords.words('english')).union(["pra"]).union(["são"])

# Function that removes stopwords
def remove_stopwords(text):
    text = text.split(' ')
    text = [x for x in text if x not in stop_words]
    return ' '.join(text)

# Function that removes words with less than 3 chars
def remove_words_less_than_2(text):
    text = text.split(' ')
    text = [x for x in text if len(x) > 2]
    return ' '.join(text)

def process_data(df):
    
    # To lowercase
    df['review'] = df['review'].str.lower()
    
    # Removing punctuations
    df['review'] = df['review'].str.replace('[^\w\s]',' ')
    
    # Remove a single character from the string
    remove_single_char = lambda x: re.sub(r"\b[a-zA-Z]\b", "", x)
    df['review'] = df['review'].apply(remove_single_char)
    
    # Substituting multiple spaces with single space
    remove_multiple_spaces = lambda x: ' '.join(x.split())
    df['review'] = df['review'].apply(remove_multiple_spaces)
    
    # Remove repeated letters if there 3 or more
    remove_repeated_letters = lambda x: re.sub(r'([a-z])\1+', r'\1\1', x)
    df['review'] = df['review'].apply(remove_repeated_letters)
    
    # removing stopwords
    rm_stopwords = lambda x: remove_stopwords(x)
    df['review'] = df['review'].apply(rm_stopwords)
    
    # Removing accents
    remove_accent = lambda x: unidecode.unidecode(x)
    df['review'] = df['review'].apply(remove_accent)
    
    # Remove words with less than 3 chars
    remove_less_2 = lambda x: remove_words_less_than_2(x)
    df['review'] = df['review'].apply(remove_less_2)
    
    return df

In [11]:
data = process_data(data)
data.head(10)

,review,voted_up
0,apenas sensacional comeco nada pro jogo poucos...,True
1,bom recomendo todos fas gostam desses tipos jogos,True
2,melhores rpg jah joguei realmente bom indico g...,True
3,super recomendado jogo jogabilidade facil temp...,True
4,jogo roda slowmotion procurando solucoes inter...,False
5,recomendaria jogo pois jogo criado fas bom dif...,True
6,gosto action rpg esforcei bastante gostar dest...,False
7,mlg pro player quickscoper 720o scope god game...,False
8,nada melhorou graficos jogo apenas ajustado po...,False
9,joguem game pois vai levar ban aleatoriamente ...,False


# TF-IDF

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

stop_words = set(stopwords.words('portuguese')).union(set(stopwords.words('english')))

corpus = data['review'].values

tfidf = TfidfVectorizer(stop_words=stop_words)
response = tfidf.fit_transform(corpus)

scores = zip(tfidf.get_feature_names(), np.asarray(response.sum(axis=0)).ravel())
sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
sorted_scores[:25]

[('jogo', 6007.948359562883),
 ('bom', 3004.1013905326463),
 ('jogar', 1730.8876919179384),
 ('recomendo', 1689.9284896992867),
 ('game', 1488.119628923753),
 ('bem', 1463.009712236176),
 ('melhor', 1426.1006082988195),
 ('nao', 1303.1224534432647),
 ('otimo', 1158.0071868889445),
 ('historia', 1150.971864884919),
 ('divertido', 1049.864144772544),
 ('ser', 1043.5040347859472),
 ('ruim', 1029.0016573043074),
 ('vale', 1004.7600212368078),
 ('jogabilidade', 996.002829511541),
 ('tempo', 968.7119154978532),
 ('legal', 926.5024268126185),
 ('pena', 922.9680262311525),
 ('jogos', 915.8141556065891),
 ('graficos', 859.225650508162),
 ('ainda', 824.3984221877075),
 ('joguei', 818.6478647821493),
 ('gostei', 774.3154620719414),
 ('amigos', 760.8766508215263),
 ('nada', 749.8783063928702)]

### Calcula a media e desvio padrão dos scores do TF-IDF

In [13]:
import statistics as st

scores_labels = [x[1] for x in sorted_scores]
media = sum(scores_labels)/len(scores_labels)
dp = st.stdev(scores_labels)

### Pega apenas os scores com valores acima da média + 1 desvio padrão

In [14]:
best_scores = []
for score in sorted_scores:
    if score[1] < media + dp:
        break
    best_scores.append(score)

### Diferença no tamanho dos conjuntos

In [15]:
print("Tamanho do conjunto original:", len(sorted_scores))
print("Tamanho do conjunto com os melhores scores:", len(best_scores))

Tamanho do conjunto original: 78726
Tamanho do conjunto com os melhores scores: 1312


## Criação do dataset utilizado no modelo de aprendizado

### Testando as melhores combinações de entrada

In [16]:
# Corpus do texto
corpus = data['review'].values
size = int(len(best_scores)/3)

# Usado para retornar o corpus apenas com as palavras top
def refined_corpus(label_scores, corpus):
    only_words = set([x[0] for x in label_scores])

    new_corpus = []
    for text in corpus:
        words = text.split(" ")
        new_text = [word for word in words if word in only_words]
        new_corpus.append(' '.join(new_text))
    
    return new_corpus

### Testando com o primeiro, segundo e terceiro terço

In [17]:
# Primeiro terço
begin = 0
end = begin + size
input_scores = best_scores[begin:end]
corpus_1_3 = refined_corpus(input_scores, corpus)

# Segundo terço
begin = size
end = begin + size
input_scores = best_scores[begin:end]
corpus_2_3 = refined_corpus(input_scores, corpus)

# Terceiro terço
begin = size * 2
end = begin + size
input_scores = best_scores[begin:end]
corpus_3_3 = refined_corpus(input_scores, corpus)

# Utilizando todos os melhores
corpus_best = refined_corpus(best_scores, corpus)

list_corpus = [
    corpus_1_3,
    corpus_2_3,
    corpus_3_3,
    corpus_best,
]


In [18]:
qts = []
for c in list_corpus:
    tfidf = TfidfVectorizer(stop_words=stop_words)
    response = tfidf.fit_transform(c)
    
    total = 0
    for review in response.toarray():
        x = sum(review)
        if x > 0:
            total += 1
    
    qts.append(total)

In [19]:
print("Corpus 1:", qts[0]/len(list_corpus[0]))
print("Corpus 2:", qts[1]/len(list_corpus[0]))
print("Corpus 3:", qts[2]/len(list_corpus[0]))
print("Full Corpus:", qts[3]/len(list_corpus[0]))

Corpus 1: 0.9332826086956522
Corpus 2: 0.6156195652173913
Corpus 3: 0.4738913043478261
Full Corpus: 0.9582826086956522


### Utilizar o primeiro terço, pois engobla 93% do dataset original

In [20]:
from sklearn.model_selection import train_test_split

tfidf = TfidfVectorizer(stop_words=stop_words)

processed_features = tfidf.fit_transform(corpus_1_3).toarray()
labels = data['voted_up'].values

X_train, X_test, y_train, y_test = train_test_split(processed_features, labels, test_size=0.2, random_state=0)

In [23]:
from sklearn.naive_bayes import MultinomialNB

nb_clf = MultinomialNB()
nb_clf.fit(X_train, y_train)
predicted = nb_clf.predict(X_test)
np.mean(predicted == y_test)

0.8009782608695653

In [22]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,predicted))
print(classification_report(y_test,predicted))
print(accuracy_score(y_test, predicted))

[[7073 2197]
 [1465 7665]]
              precision    recall  f1-score   support

       False       0.83      0.76      0.79      9270
        True       0.78      0.84      0.81      9130

    accuracy                           0.80     18400
   macro avg       0.80      0.80      0.80     18400
weighted avg       0.80      0.80      0.80     18400

0.8009782608695653
